# Databricks Host and Token
### The Personal Access Token is used to authenticate your SDK calls. It's a secret key, so treat it like a password!
1. In your Databricks workspace, click your username in the top-right corner.
2. Select Settings.
3. Click the Developer tab.
4. Next to Access tokens, click Manage.
5. Click the Generate new token button.
6. Enter a Comment (e.g., "SDK Access") and set a Lifetime for the token (the recommended practice is to set an expiration date).
7. Click Generate.
8. Immediately copy the displayed token. This is the only time Databricks will show you the token. If you lose it, you'll have to generate a new one.
9. To set the on linux or a mac run...  export DATABRICKS_HOST="https://<your-workspace-url>". and export DATABRICKS_TOKEN="dapi****************************"

In [2]:
import os
from dotenv import load_dotenv
from databricks.sdk import WorkspaceClient

from ag_vision.mobile.ingest import MobileImageIngest

# Load environment variables
load_dotenv(".env")

host = os.environ["DATABRICKS_HOST"]
token = os.environ["DATABRICKS_TOKEN"]

w = WorkspaceClient(host=host, token=token)

In [ ]:
p_path = '/Users/danielwilliams/Documents/project data/board data/'
files = os.listdir(p_path)
files = [x for x in files if '.jpeg' in x]
files = [p_path + x for x in files]
files

In [ ]:
# create an empty MobileImageIngest object.
ingest = MobileImageIngest(platform='local',
                           cloud_bucket=None,
                           ingest_df=None)

In [ ]:
# generate the ingest_df from the list of files you want to upload.
ingest.generate_ingest_df(file_list=files)

In [ ]:
ingest.ingest_df.head()

In [ ]:
# generate unique ID's for each of the images. This method used UUID4()
ingest.generate_unique_image_ids()

In [ ]:
# these are all the columns that you need to upload mobile scouting data to Fairgorunds and follow the Data Architecture Laid out in ag_vision.
ingest.ingest_df['project_dir'] = '/Volumes/ue1_prod_catalog_119738067017277/dan_test/test_images'
ingest.ingest_df['site'] = 'California-labs'
ingest.ingest_df['trial'] = 'grain-analysis'
ingest.ingest_df['year'] = 2025
ingest.ingest_df['country'] = 'USA'
ingest.ingest_df['crop'] = 'corn'
ingest.ingest_df['time_of_year'] = 'spring'
ingest.ingest_df['field'] = 'home'
ingest.ingest_df['location'] = 'home_1'
ingest.ingest_df['task'] = 'grain_imaging'
ingest.ingest_df['protocol'] = 'grain_image_blue_board'
ingest.ingest_df['event_type'] = 'scouting'
ingest.ingest_df['collection_date'] = '11/12/2025'
ingest.ingest_df['plot_id'] = 'none' # if you want to upload event_type of trial you will need plot ID's for each image.
ingest.ingest_df['img_ext'] = '.jpg'

In [ ]:
ingest.ingest_df.head()

In [ ]:
# From the year, country, crop, and time of year we can generate a season code.
ingest.add_season_column()

In [ ]:
# Validate that all the columns are there and for some columns valdiate what is in them.
ingest.validate_ingest_df()

In [ ]:
# generate the dst_path from all the data in the column of ingest_df.
ingest.generate_dst_path_name()

In [ ]:
ingest.ingest_df.head()

In [ ]:
# this uploads the data to the cloud. If files failed they will be stored in the failed_upload list.
ingest.upload_local_data_to_db(db_client=w)